In [9]:
import os
import shutil
import random

data_dir = 'dogsandcats'
train_dir = data_dir  # 原始训练集目录
test_dir = os.path.join(data_dir, 'test')

# 创建测试集
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# 获取所有图片文件名
all_images = [img for img in os.listdir(train_dir) if img.startswith('cat') or img.startswith('dog')]

# 随机选5000张图片作为测试集
test_images = random.sample(all_images, 5000)

# 直接剪切图片
for img in test_images:
    shutil.move(os.path.join(train_dir, img), os.path.join(test_dir, img))

print('5000张图片已随机抽取并移动到测试集目录')


5000张图片已随机抽取并移动到测试集目录


In [10]:
from torch.utils.data import Dataset
from PIL import Image
import os

class CatsAndDogsDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = []
        self.labels = []

        for file_name in os.listdir(folder_path):
            if file_name.startswith('cat'):
                label = 0
            elif file_name.startswith('dog'):
                label = 1
            else:
                continue
            
            self.images.append(file_name)
            self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path, self.images[idx])
        image = Image.open(img_name)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [11]:
from torch.utils.data import DataLoader
from torchvision import transforms

# 定义数据增强和预处理
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_dir = './dogsandcats/test'  # 测试集
test_dataset = CatsAndDogsDataset(folder_path=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [12]:
import torch
from torchvision import models

model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 2)  # 替换最后一层，全连接层输出2个类

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 加载模型
model.load_state_dict(torch.load('cats_and_dogs_resnet18.pth'))
model.eval()  # 切到评估模式


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return torch.sum(preds == labels).item() / len(labels)

model.eval()
test_corrects = 0
total_test_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        test_corrects += calculate_accuracy(outputs, labels) * labels.size(0)
        total_test_samples += labels.size(0)

test_accuracy = test_corrects / total_test_samples
print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 0.9755
